
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [55]:
import pandas as pd
import numpy as np
from scipy import stats as st

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [47]:
#We are going to compare the average callback rate for black applicants with the average callback rate for white applicants
#The Central Limit theorem applies here, as each group has n=2435>30
#The test we are performing is a Two Sample proportion comparison test using the Z test statistic
black = data[data.race =='b']
white = data[data.race =='w']
n1 = len(white)
n2 = len(black)
n1

2435

In [43]:
#Null Hypothesis: The proportion of black applicants who received callbacks = proportion of white applicants who received callbacks
# (p1 - p2 = 0)
#Alt Hypothesis: The proportion of black applicants who received callbacks != proportion of white applicants who received callbacks
# (p1 - p2 != 0)
p2 = (black.call).mean()
#157/2435 == 0.0645 == 6.45%

In [44]:
p1 = (white.call).mean()
#235/2435 == 0.0965 == 9.65%

In [46]:
#difference in proportions: white - black = 0.0965 - 0.0645 = 0.032
d = p1 - p2
#P is pooled successes over total n
P = (157 + 235)/(2435 + 2435)
P

0.08049281314168377

In [50]:
#Z test statistic forumla for two proportion test
Z = (d - 0) / np.sqrt((P)*(1-P)*((1/n1)+(1/n2)))
Z

4.1084122352384718

In [61]:
#We are performing a two sided proportion Z test, so I solved for the area under the cdf to the right of Z, times 2 for both tails
pval = (1 - st.norm.cdf(Z)) *2
pval
#pval = .000039839 < .01 < .05
#Therefore, we reject the null hypothesis that the proportion of black applicants who received callbacks is equal to the
#proportion of white applicants who received callbacks, and conclude that the true proportion of applicants who are called back
#is different for black and white applicants

3.9838854094975673e-05

In [72]:
#I will also construct a 95% confidence interval for the difference between callback rate of white and black applicants
SE = np.sqrt((P)*(1-P)*((1/n1)+(1/n2)))
#This is formula for non-pooled 
#SE = np.sqrt((p1*(1-p1))/n1 + (p2*(1-p2))/n2)
SE
#In this situation, we should continue to use the pooled standard error becuase H0: p1-p2 = 0
#Either way, the difference in this case is negligible(no change in first 4 decimal places)
ME = SE*1.96
ME

0.015281912310894095

In [75]:
pd.DataFrame([d-ME,d+ME], index=['Lower Limit','Upper Limit'], columns = ["95% Confidence Interval for Difference in Callback Rates"])

,95% Confidence Interval for Difference in Callback Rates
Lower Limit,0.016751
Upper Limit,0.047315


In [ ]:
#The above confidence interval shows us that the difference between the two sample proportions would fall within 0.016751 and 0.047315
#in 95% of replicated trials of this hypothesis test. Because 0 is not in this interval, we would expect replicated trials to find
#that there is a difference between the proportion of callbacks for black and white applicants.